In [1]:

# input, output and command line tools
import os
from os.path import isdir, join
import pandas as pd

#math and data handler
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

# audio file i/o
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile

In [2]:
#Visualization
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

mpl.rc('font', family = 'serif', size = 17)
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.minor.size'] = 2
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.minor.size'] = 2

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [3]:
data_dir = '../data/train/audio'

## change the name of `_background_noise_' into 'silence` which is a proper label name
if os.path.exists(data_dir + '/' + '_background_noise_'):
    os.system('mv {0}/_background_noise_ {1}/silence'.format(data_dir, data_dir))
if os.path.exists(data_dir + '/' + 'silence/README.md'):
     os.system('rm {0}/silence/README.md'.format(data_dir))

In [4]:
labels = os.listdir(data_dir)
print(labels, len(labels))

(['house', 'off', 'yes', 'happy', 'zero', 'six', 'silence', 'tree', 'stop', 'four', 'marvin', 'seven', 'cat', 'dog', 'up', 'down', 'one', 'sheila', 'bird', 'on', 'left', 'eight', 'five', 'nine', 'two', 'three', 'go', 'no', 'bed', 'wow', 'right'], 31)


In [5]:
def load_audio_data(path):
    '''
    path: audio file path
    return: pd.DataFrame
    '''
    raw = {'x': [], 'y': [], 'label':[]}
    for i, folder in enumerate(os.listdir(path)):
        for filename in os.listdir(path + '/' + folder):
            rate, sample = wavfile.read(data_dir + '/' + folder + '/' + filename)
            assert(rate == 16000)
            p = max(0, rate - len(sample))
            sample = np.pad(sample, [(0,p)], mode='constant')
            sample = sample[:rate]
            raw['x'].append(np.array(sample))
            raw['y'].append(i)
            raw['label'].append(folder)
    return pd.DataFrame(raw)
%time raw_df = load_audio_data(data_dir)

/opt/anaconda2/lib/python2.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning:

Chunk (non-data) not understood, skipping it.



CPU times: user 5.71 s, sys: 2.2 s, total: 7.91 s
Wall time: 7.91 s


In [6]:
# show the data Frame
display(raw_df[:5])

,label,x,y
0,house,"[-173, -197, -194, -216, -201, -275, -255, -24...",0
1,house,"[6, -56, -4, 8, -18, 76, 49, 24, 59, 42, 54, 5...",0
2,house,"[10, 10, 17, 19, 27, 19, 4, 11, 10, 17, 16, 17...",0
3,house,"[32, 63, 86, 79, 89, 109, 76, 122, 92, 18, 65,...",0
4,house,"[-83, -62, -93, -71, -16, -3, 1, -43, -6, -96,...",0


In [7]:
from sklearn.utils import shuffle
def train_test_split(df, ratio = 0.7):
    '''
    return train_sets + test_sets + label_map, which maps from y to label name
    '''
    test_x = []
    test_y = []
    train_x = []
    train_y = []
    label_map = {}
    for i in set(df.y.tolist()):
        tmp_df = df[df.y == i]
        label_map[i] = tmp_df.label.tolist()[0]
        tmp_df = shuffle(tmp_df)
        tmp_n = int(len(tmp_df)*ratio)
        train_x += tmp_df.x.tolist()[: tmp_n]
        test_x += tmp_df.x.tolist()[tmp_n: ]
        train_y += tmp_df.y.tolist()[: tmp_n]
        test_y += tmp_df.y.tolist()[tmp_n: ]
    return np.array(train_x), np.array(train_y), np.array(test_x), np.array(test_y), label_map

In [8]:
# Getting testing and trainning set
%time tr_x, tr_y, ts_x, ts_y, idmap = train_test_split(raw_df, ratio=0.7)
print(np.shape(tr_x))
print(np.shape(tr_y))
print(np.shape(ts_x))
print(np.shape(ts_y))
print(type(tr_x), np.shape(tr_x[0]))
print(idmap)
print len(tr_x)

CPU times: user 416 ms, sys: 480 ms, total: 896 ms
Wall time: 895 ms
(45296, 16000)
(45296,)
(19431, 16000)
(19431,)
(<type 'numpy.ndarray'>, (16000,))
{0: 'house', 1: 'off', 2: 'yes', 3: 'happy', 4: 'zero', 5: 'six', 6: 'silence', 7: 'tree', 8: 'stop', 9: 'four', 10: 'marvin', 11: 'seven', 12: 'cat', 13: 'dog', 14: 'up', 15: 'down', 16: 'one', 17: 'sheila', 18: 'bird', 19: 'on', 20: 'left', 21: 'eight', 22: 'five', 23: 'nine', 24: 'two', 25: 'three', 26: 'go', 27: 'no', 28: 'bed', 29: 'wow', 30: 'right'}
45296


In [9]:
def fft_convert(samples, rate = 16000, n = 25, m = 16, NR = 256, NC = 128, delta = 1.E-10):
    '''
    convert input data into a big spectrum matrix
    '''
    res = []
    for i,sam in enumerate(samples):
        if(i % 5000 == 0):
            print(i)
        freq, times, spec = signal.spectrogram(sam, fs=rate, window=('kaiser',10), nperseg=int(n*rate/1000),
                                               noverlap=int(m*rate/1000))
        p1 = max(0, NR - np.shape(spec)[0])
        p2 = max(0, NC - np.shape(spec)[1])
        spec = np.pad(spec, [(0,p1), (0, p2)], mode='constant')
        spec = spec[:NR, :NC]
        res.append(spec)
    return np.log(np.array(res) + delta)

'''%time train_x = fft_convert(tr_x)
%time test_x = fft_convert(ts_x)

print np.shape(train_x), np.shape(test_x)'''

'%time train_x = fft_convert(tr_x)\n%time test_x = fft_convert(ts_x)\n\nprint np.shape(train_x), np.shape(test_x)'

In [10]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.utils import np_utils, plot_model
from keras.layers.recurrent import SimpleRNN, LSTM #Actually in this test, SimpleRNN works much better
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [11]:
n_cls = 31
tr_y = np_utils.to_categorical(tr_y, n_cls)
ts_y = np_utils.to_categorical(ts_y, n_cls)
print np.shape(tr_y), np.shape(ts_y)
print len(tr_x), len(ts_x)

(45296, 31) (19431, 31)
45296 19431


In [12]:
model = Sequential()
model.add(LSTM(512, input_shape = (None, 1)))
model.add(Activation('relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(n_cls, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               1052672   
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_2 (Dense)              (None, 31)                3999      
Total params: 1,122,335.0
Trainable params: 1,122,335.0
Non-trainable params: 0.0
_________________________________________________________________


In [13]:
### Compile the model
optimizer = SGD()
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [14]:
### Train the model
n_tr = len(tr_x)
n_ts = len(ts_x)
train_x = tr_x.reshape(n_tr, 16000, 1)
test_x = ts_x.reshape(n_ts, 16000, 1)
res = model.fit(train_x, tr_y, batch_size = 128, epochs = 12, verbose = 1, validation_data = (test_x, ts_y))

Train on 45296 samples, validate on 19431 samples
Epoch 1/12


ResourceExhaustedError: OOM when allocating tensor with shape[2048000,512]
	 [[Node: lstm_1/MatMul_3 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](lstm_1/Reshape_6, lstm_1/strided_slice_3)]]
	 [[Node: Mean_3/_31 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2284_Mean_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'lstm_1/MatMul_3', defined at:
  File "/opt/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/opt/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/opt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/anaconda2/lib/python2.7/site-packages/traitlets/